# CBLB cohort
Data from 
[Janssen E, et al. (2022) Immune dysregulation caused by homozygous mutations in CBLB. J Clin Invest 132(20):e154487](https://pubmed.ncbi.nlm.nih.gov/36006710/).

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) 
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.42


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [3]:
df = pd.read_excel("input/CBLB_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,Comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Type I diabetes mellitus,Vitiligo,Urticaria,Recurrent otitis media,Recurrent lower respiratory tract infections,Abnormal T cell count,Abnormal B cell count,Abnormal natural killer cell count,Hepatic hemophagocytosis,Recurrent fever
0,str,str,str,optional str,str,str,str,NaN,NaN,optional str,...,HP:0100651,HP:0001045,HP:0001025,HP:0000403,HP:0002783,HP:0011839,HP:0010975,HP:0040089,HP:0034797,HP:0001954
1,PMID:36006710,Immune dysregulation caused by homozygous mutations in CBLB,Patient 1,NaN,OMIM:620430,"Autoimmune disease, multisystem, infantile-onset, 3",NM_170662.5,c.770A>T,c.770A>T,NP_733762.2:p.(His257Leu); in original publication c.854A>T:p.H285L (NM_001321786.1).,...,P6Y,P6Y,observed,observed,observed,excluded,excluded,excluded,excluded,excluded


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)

Created encoders for 32 fields


In [5]:
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="CBLB",
                          transcript="NM_170662.5",
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,1,c.770A>T
1,unmapped,0,


In [7]:
individuals = encoder.get_individuals()
vmanager.add_variants_to_individuals(individuals)
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [8]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Patient 1 (FEMALE; P1Y),"Autoimmune disease, multisystem, infantile-onset, 3 (OMIM:620430)",NM_170662.5:c.770A>T (homozygous),P1Y: Hypothyroidism (HP:0000821)P6Y: Type I diabetes mellitus (HP:0100651); Vitiligo (HP:0001045)Decreased circulating IgG level (HP:0004315); Decreased circulating IgA level (HP:0002720); Decreased circulating total IgM (HP:0002850); Partial absence of specific antibody response to tetanus vaccine (HP:0410297); Decreased specific pneumococcal antibody level (HP:0012476); Increased circulating interleukin 10 concentration (HP:0033199); Increased circulating interleukin 6 concentration (HP:0030783); Urticaria (HP:0001025); Recurrent otitis media (HP:0000403); Recurrent lower respiratory tract infections (HP:0002783); excluded: Autoimmune hemolytic anemia (HP:0001890); excluded: Abnormal T cell count (HP:0011839); excluded: Abnormal B cell count (HP:0010975); excluded: Abnormal natural killer cell count (HP:0040089); excluded: Hepatic hemophagocytosis (HP:0034797); excluded: Recurrent fever (HP:0001954)
Patient 2 (MALE; P3Y),"Autoimmune disease, multisystem, infantile-onset, 3 (OMIM:620430)",NM_170662.5:c.770A>T (homozygous),Increased circulating interleukin 10 concentration (HP:0033199); Increased circulating interleukin 6 concentration (HP:0030783); Hypothyroidism (HP:0000821); Type I diabetes mellitus (HP:0100651); Hepatic hemophagocytosis (HP:0034797); Recurrent fever (HP:0001954); excluded: Autoimmune hemolytic anemia (HP:0001890); excluded: Decreased circulating IgG level (HP:0004315); excluded: Decreased circulating IgA level (HP:0002720); excluded: Decreased circulating total IgM (HP:0002850); excluded: Partial absence of specific antibody response to tetanus vaccine (HP:0410297); excluded: Vitiligo (HP:0001045); excluded: Urticaria (HP:0001025); excluded: Recurrent otitis media (HP:0000403); excluded: Recurrent lower respiratory tract infections (HP:0002783); excluded: Abnormal T cell count (HP:0011839); excluded: Abnormal B cell count (HP:0010975); excluded: Abnormal natural killer cell count (HP:0040089)
Patient 3 (MALE; P6M),"Autoimmune disease, multisystem, infantile-onset, 3 (OMIM:620430)",NM_170662.5:c.770A>T (homozygous),Autoimmune hemolytic anemia (HP:0001890); Decreased circulating total IgM (HP:0002850); Decreased specific pneumococcal antibody level (HP:0012476); Increased circulating interleukin 10 concentration (HP:0033199); Increased circulating interleukin 6 concentration (HP:0030783); excluded: Decreased circulating IgG level (HP:0004315); excluded: Decreased circulating IgA level (HP:0002720); excluded: Partial absence of specific antibody response to tetanus vaccine (HP:0410297); excluded: Hypothyroidism (HP:0000821); excluded: Type I diabetes mellitus (HP:0100651); excluded: Vitiligo (HP:0001045); excluded: Urticaria (HP:0001025); excluded: Recurrent otitis media (HP:0000403); excluded: Recurrent lower respiratory tract infections (HP:0002783); excluded: Abnormal T cell count (HP:0011839); excluded: Abnormal B cell count (HP:0010975); excluded: Abnormal natural killer cell count (HP:0040089); excluded: Hepatic hemophagocytosis (HP:0034797); excluded: Recurrent fever (HP:0001954)


In [10]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 3 GA4GH phenopackets to the directory phenopackets


In [9]:
builder = HpoaTableBuilder.from_individuals(individuals, created_by=created_by)
pmid = "PMID:36006710"
builder.infantile_onset(pmid,1,3).childhood_onset(pmid,2,3)
builder.autosomal_recessive(pmid)
hpoa_creator = builder.build()

We found a total of 19 unique HPO terms
Extracted disease: Autoimmune disease, multisystem, infantile-onset, 3 (OMIM:620430)


In [11]:
df = hpoa_creator.get_dataframe()
df.head()

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620430,"Autoimmune disease, multisystem, infantile-onset, 3",HP:0004315,Decreased circulating IgG level,,,1/3,,,,,PMID:36006710,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620430,"Autoimmune disease, multisystem, infantile-onset, 3",HP:0002720,Decreased circulating IgA level,,,1/3,,,,,PMID:36006710,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620430,"Autoimmune disease, multisystem, infantile-onset, 3",HP:0002850,Decreased circulating total IgM,,,2/3,,,,,PMID:36006710,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620430,"Autoimmune disease, multisystem, infantile-onset, 3",HP:0410297,Partial absence of specific antibody response to tetanus vaccine,,,1/3,,,,,PMID:36006710,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620430,"Autoimmune disease, multisystem, infantile-onset, 3",HP:0012476,Decreased specific pneumococcal antibody level,,,2/2,,,,,PMID:36006710,PCS,ORCID:0000-0002-0736-9199


In [12]:
hpoa_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620430.tab
